In [1]:
import os
import glob
import numpy as np
import datetime as dt
from multiprocessing import Pool
from tqdm.notebook import tqdm
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import matplotlib.pyplot as plt
from grid_age import GridAge


In [2]:
land_50m = cfeature.LAND
sid_srs = ccrs.LambertAzimuthalEqualArea(central_longitude=0, central_latitude=-90)

In [13]:
force = False
lag_dir = r"D:/SIA_Weddell_Sea"
srd_dir = f'{lag_dir}/age'

mesh_init_file = r"D:/mesh_weddel_ease_25km_mmg_40_52.npz"
xc = np.load(mesh_init_file)['xc']
yc = np.load(mesh_init_file)['yc'][::-1]
mask = np.load(mesh_init_file)['mask']
xgrd, ygrd = np.meshgrid(xc, yc)

sic = np.load(r"D:/OSI-SAF/Preprocessing_Antarctica/2015/ice_drift_sh_ease2-750_cdr-v1p0_24h-201501011200.nc.npz")['c']
landmask = np.isnan(sic).astype(float)
landmask[landmask == 0] = np.nan

In [14]:
age_files = sorted(glob.glob(f'{srd_dir}/*/age_????????.npz'))
print(len(age_files), age_files[0], age_files[-1])

304 D:/SIA_Weddell_Sea/age\2015\age_20150301.npz D:/SIA_Weddell_Sea/age\2015\age_20151231.npz


In [15]:
f = r"D:/SIA_Weddell_Sea/unc/2015/unc_tot_20150301.npz"
with np.load(f) as data:
    print("Keys:", data.files)


Keys: ['unc_tot', 'unc_age']


In [16]:
grid_age = GridAge(xgrd, ygrd, mask, force=force)

In [17]:
with Pool(4) as p:
    r = list(tqdm(p.imap(grid_age, age_files), total=len(age_files)))

  0%|          | 0/304 [00:00<?, ?it/s]

In [16]:
age_file_error = np.load("D:/SIA_Weddell_Sea/age/2015/age_20151109.npz")

print("FILES IN NPZ:", age_file_error.files)

# Loop through all arrays inside the npz and print info
for key in age_file_error.files:
    arr = age_file_error[key]

    print(f"\n--- ARRAY: {key} ---")
    print("Type:", type(arr))
    print("Shape:", arr.shape)
    print("dtype:", arr.dtype)

    if np.issubdtype(arr.dtype, np.number):
        print("NaNs:", np.isnan(arr).sum())
        print("Unique values:", len(np.unique(arr)))
    else:
        print("Non-numeric array → skipping NaN/unique checks")


FILES IN NPZ: ['a', 'c', 'x', 'y', 't', 'f']

--- ARRAY: a ---
Type: <class 'numpy.ndarray'>
Shape: (11021,)
dtype: float32
NaNs: 0
Unique values: 6622

--- ARRAY: c ---
Type: <class 'numpy.ndarray'>
Shape: (11021,)
dtype: float32
NaNs: 0
Unique values: 5391

--- ARRAY: x ---
Type: <class 'numpy.ndarray'>
Shape: (5652,)
dtype: float32
NaNs: 0
Unique values: 5651

--- ARRAY: y ---
Type: <class 'numpy.ndarray'>
Shape: (5652,)
dtype: float32
NaNs: 0
Unique values: 5651

--- ARRAY: t ---
Type: <class 'numpy.ndarray'>
Shape: (11021, 3)
dtype: float32
NaNs: 0
Unique values: 5652

--- ARRAY: f ---
Type: <class 'numpy.ndarray'>
Shape: (2, 11021)
dtype: float32
NaNs: 0
Unique values: 13226


In [20]:
# make monthly averages
age_dir = r"D:\SIA_Weddell_Sea\age"

for year in range(2015,2015):
    for month in range(1,13):
        ofile = f'{age_dir}/age_{year:04d}{month:02d}01_grd.npz'
        if os.path.exists(ofile):
            print(f'Skipping {ofile} as it already exists.')
            continue

        year_dir = f'{age_dir}/{year:04d}'
        month_files = sorted(glob.glob(f'{year_dir}/age_{year:04d}{month:02d}*.npz'))
        if len(month_files) == 0:
            print(f'No files found for {year:04d}-{month:02d}. Skipping.')
            continue
        print(len(month_files), month_files[0], month_files[-1])
        age_data = [dict(**np.load(f)) for f in month_files]

        max_fractions = []
        for data in age_data:
            max_fraction = max([int(key.split('_')[1].replace('yi','')) for key in age_data[0].keys() if 'fraction' in key and 'yi' in key])
            max_fractions.append(max_fraction)
        max_fraction = min(max_fractions)    

        keys = ['age'] + [f'fraction_{i}yi' for i in range(1, max_fraction + 1)]
        unc_keys = [f'{key}_unc' for key in keys]
        avg_data = {}
        for data in age_data:
            for key in keys:
                if key not in avg_data:
                    avg_data[key] = 0
                else:
                    avg_data[key] += data[key]
            for key in unc_keys:
                if key not in avg_data:
                    avg_data[key] = 0
                else:
                    avg_data[key] += data[key]**2

        for key in avg_data:
            avg_data[key] /= len(age_data)
            if '_unc' in key:
                avg_data[key] = np.sqrt(avg_data[key])       

        np.savez(ofile, **avg_data)        